In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
pip install transformers

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('nlp-3 training data.csv', encoding="utf-8")
#input_csv = 'nlp-3 training data.csv'
dataset = Dataset.from_pandas(df)

In [3]:
df.columns = [col.lower() for col in df.columns]
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

In [4]:
#convert_columns_to_lowercase(input_csv)
df


,questions,labels
0,where can i find the crème de la mer document?,moisturizer/la mer
1,show me the file for crème de la mer.,moisturizer/la mer
2,i need the data on crème de la mer.,moisturizer/la mer
3,locate the crème de la mer information.,moisturizer/la mer
4,find the report on crème de la mer.,moisturizer/la mer
...,...,...
751,how can i locate the moisture bound sleeping r...,face mask/amorepacific
752,where can i find the moisture bound sleeping r...,face mask/amorepacific
753,what is the directory for the moisture bound s...,face mask/amorepacific
754,how do i access the moisture bound sleeping re...,face mask/amorepacific


In [5]:
#df['filtered_questions']

In [6]:

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
unique_labels = df['labels'].unique()
label_encoder.fit(unique_labels)
df['label_encoded'] = label_encoder.transform(df['labels'])
encoded_labels = label_encoder.fit_transform(unique_labels)

In [7]:
def tokenize_function(examples):
    return tokenizer(examples['questions'], padding='max_length', truncation=True)


In [8]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_data = df['questions'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=128))
df['input_ids'] = tokenized_data.apply(lambda x: x['input_ids'])
df['attention_mask'] = tokenized_data.apply(lambda x: x['attention_mask'])
df.head()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,questions,labels,label_encoded,input_ids,attention_mask
0,where can i find the crème de la mer document?,moisturizer/la mer,28,"[101, 2073, 2064, 1045, 2424, 1996, 13675, 213...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
1,show me the file for crème de la mer.,moisturizer/la mer,28,"[101, 2265, 2033, 1996, 5371, 2005, 13675, 213...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
2,i need the data on crème de la mer.,moisturizer/la mer,28,"[101, 1045, 2342, 1996, 2951, 2006, 13675, 213...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
3,locate the crème de la mer information.,moisturizer/la mer,28,"[101, 12453, 1996, 13675, 21382, 2139, 2474, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
4,find the report on crème de la mer.,moisturizer/la mer,28,"[101, 2424, 1996, 3189, 2006, 13675, 21382, 21...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [9]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(df['labels'])))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("labels", "labels")


Map:   0%|          | 0/756 [00:00<?, ? examples/s]

KeyError: 'questions'

In [10]:
import torch
from torch.utils.data import Dataset

class FAQDataset(Dataset):
    def __init__(self, df):
        self.input_ids = list(df['input_ids'])
        self.attention_mask = list(df['attention_mask'])
        self.labels = list(df['label_encoded'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }
dataset = FAQDataset(df)

In [11]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = FAQDataset(train_df)
val_dataset = FAQDataset(val_df)

In [12]:

from torch.utils.data import DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
pip install -U accelerate

In [25]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9984739a9a64f6f859b3312c9677502ceae28fead303de6fe986b2695ac6df53
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [23]:
!pip install datasets
!pip install scikit-learn

In [26]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_metric
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    accuracy = accuracy_score(p.label_ids, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    rouge = load_metric("rouge")
    rouge_score = rouge.compute(predictions=preds, references=p.label_ids)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'rouge1': rouge_score['rouge1'].mid.fmeasure,
        'rouge2': rouge_score['rouge2'].mid.fmeasure,
        'rougeL': rouge_score['rougeL'].mid.fmeasure
    }
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    logging_strategy='steps',  # Set logging strategy to steps
    logging_steps=10,          # Log every 10 steps (adjust as needed)
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.05,
    report_to="none",          # Disable reporting to external services
    save_strategy="epoch"
)

# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Rouge1,Rouge2,Rougel
1,3.010500,2.465450,0.414474,0.297943,0.414474,0.298697,0.414474,0.000000,0.414474
2,1.806900,1.553363,0.644737,0.540433,0.644737,0.562197,0.644737,0.000000,0.644737
3,1.300200,1.193042,0.710526,0.657271,0.710526,0.649359,0.710526,0.000000,0.710526


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in 

TrainOutput(global_step=114, training_loss=2.191743827702706, metrics={'train_runtime': 43.4337, 'train_samples_per_second': 41.719, 'train_steps_per_second': 2.625, 'total_flos': 60051607852032.0, 'train_loss': 2.191743827702706, 'epoch': 3.0})

In [14]:
import pickle
model_file_path = 'distilbert_model.pkl'
with open(model_file_path, 'wb') as f:
    pickle.dump((model.state_dict(), model.config), f)

print(f"Model saved to {model_file_path}")

# Load the model using pickle
with open(model_file_path, 'rb') as f:
    state_dict, config = pickle.load(f)

# Recreate the model using the loaded state_dict and config
loaded_model = AutoModelForSequenceClassification.from_config(config)
loaded_model.load_state_dict(state_dict)

Model saved to distilbert_model.pkl


<All keys matched successfully>

In [16]:
pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
results = trainer.evaluate()

# Print the evaluation results
print(results)

{'eval_loss': 1.112873911857605, 'eval_runtime': 0.6436, 'eval_samples_per_second': 236.169, 'eval_steps_per_second': 15.537, 'epoch': 3.0}


In [17]:
def predict_top_3(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    top_3_indices = torch.topk(probabilities, 3, dim=1).indices[0].cpu().numpy()
    top_3_labels = label_encoder.inverse_transform(top_3_indices)
    top_3_probs = probabilities[0, top_3_indices].cpu().numpy()
    return list(zip(top_3_labels, top_3_probs))

def print_folders_and_subfolders(predictions):
    for label, prob in predictions:
        folder, sub_folder = label.split('/')
        print(f"Folder: {folder}, Sub-folder: {sub_folder}, Probability: {prob:.4f}")

example_text = "where is the vitamin glowing face mask located?"
predictions = predict_top_3(example_text)
print_folders_and_subfolders(predictions)

Folder: face mask, Sub-folder: fresh, Probability: 0.7897
Folder: eye cream, Sub-folder: fresh, Probability: 0.0319
Folder: moisturizer, Sub-folder: fresh, Probability: 0.0210
